In [1]:
import pandas as pd
import numpy as np

## My F-beta score

Difference with default F1 score:
When deal with tricky metric like this
<pre>
|         |      |     True    |
|         |      |   0  |   1  |
|---------|------|------|------|
| Predict |   0  |   3  |   0  |
|         |   1  |   4  |   0  |
</pre>
No real positive samples makes the denominator and numeratore of F1 metric equal to 0. F1, recall, precision = 0/0.
Default F1 test are all equal to 0. But in my opinion: F1, recall, precision score are 0, 1, 0 respectively.

-------
And this:
          True
           0
Predict   0   3
In this case, I will set all of F1, recall, precision to 1. However, the default is 0.

--------
Besides, my version could return the cross table as above.

--------

Note about Beta:
![title](f1score.svg)
beta < 1 lends more weight to precision, while beta > 1 favors recall

In [2]:
def Score (PredictList,TrueList,beta):
    X=pd.concat([pd.DataFrame(PredictList),pd.DataFrame(TrueList)],axis=1, ignore_index=True)
    X['count']=1
    X.columns=['Predict','True','count']
    Acc=X.pivot_table(index='Predict',columns='True',values='count',aggfunc=np.sum)
    Acc=Acc.fillna(value=0)
    rate,f1,recall,precision=0,0,0,0
    b=beta**2
    if Acc.shape==(2,2):
        rate=rate+(float(Acc[0][0]+Acc[1][1])/Acc.sum().sum())
        if Acc[1][1]!=0:
            recall=float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])
            precision=float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])
            f1=(1+b) * (float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) * (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])) / ((b * float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) + (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])))
    elif Acc.shape==(1,2):
        if Acc.index[0]==0:
            rate=rate+(float(Acc[0][0])/Acc.sum().sum())
        else:
            rate=float(Acc[0][1])/Acc.sum().sum()
            recall=1
            precision=float(Acc[0][1])/Acc.sum().sum()
            f1=(1+b) * ((float(Acc[0][1])/Acc.sum().sum()) * 1) / ((b * float(Acc[0][1])/Acc.sum().sum()) + 1)
    elif Acc.shape==(2,1):
        if Acc.columns==1:
            rate=float(Acc.iloc[1])/Acc.sum().sum()
            recall=float(Acc.iloc[1])/Acc.sum().sum()
            precision=1
            f1=(1+b) * float(Acc.iloc[1])/Acc.sum().sum()/(float(Acc.iloc[1])/Acc.sum().sum()+ b)
        else:
            rate=float(Acc.iloc[0])/Acc.sum().sum()
            recall=1
    else: #Acc.shape=(1,1)
        rate=1
        recall=1
        precision=1
        f1=1
    return Acc,rate,f1,recall,precision

# Import Y

In [3]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)

In [7]:
Y=Y[Y.index.isin(new_metric.index)]

In [5]:
Y['Local'].sum()

109L

In [8]:
len(Y)

2692

# Histgram of Commenters

In [45]:
#page-unique_user
like_unique=likeall.groupby(['pageid','uid']).mean()

In [63]:
comm_unique=commall.groupby(['pageid','uid']).mean()

In [77]:
print len(likeall.pageid.unique())
print len(commall.pageid.unique())

1511
1452


In [66]:
a2=comm_unique.index.tolist()
b2=[i[0] for i in a2]
unique2,counts2=np.unique(b2, return_counts=True)
count2=dict(zip(unique2, counts2))
count_2=count2.values()

In [46]:
a=like_unique.index.tolist()

In [47]:
#replicable pageid
b=[i[0] for i in a]

In [48]:
#count={page: count of unique user}
unique, counts = np.unique(b, return_counts=True)
count=dict(zip(unique, counts))

In [49]:
#list of unique users number
count_=count.values()

In [135]:
print values
print base[0]

[48  0  0 49 34  0 74 24 28 52 56 39 35 59 41 51 63 54 51 59 64 64 65 58 47
 43 43 28 32 42 25 21 24 15  8 17  5  9  4  3  4  3  4  4  1  0  1  1  0]
1.0


In [138]:
print cumulative[0:5]
print base[0:5]

[0.03305785123966942, 0.03305785123966942, 0.03305785123966942, 0.06680440771349862, 0.09022038567493113]
[ 1.          1.25892541  1.58489319  1.99526231  2.51188643]


In [164]:
math.log(values[0])

3.871201010907891

In [83]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import math

fig=plt.figure(figsize=(20,15))
ax=fig.add_subplot(111)
xlabs=[10**i for i in np.arange(0,7,0.1)]
# evaluate the histogram
values, base = np.histogram(count_, bins=xlabs)
values2, base2 = np.histogram(count_2, bins=xlabs)
#evaluate the cumulative
cumulative = np.cumsum(values)
cumulative2 = np.cumsum(values2)
# plot the cumulative function
cumulative=[float(i)/1511 for i in cumulative]
cumulative2=[float(i)/1511 for i in cumulative2]
#values=[math.log(i+1) for i in values]
#ax.plot(base[:-1], cumulative, c='blue')
#ax.loglog(base[:-1], cumulative, basex=10, basey=10)
ax.plot(base[:-1], cumulative,c='blue',lw=2, label='comment')
ax.plot(base2[:-1], cumulative2,c='green',lw=2, label='like')
plt.legend(loc='lower right')
ax.plot()
ax.set_xscale("log", nonposx='clip')
ax.set_yscale("log", nonposx='clip')
ax.set_ylim(10**(-3),1.5)
plt.xlabel('count of unique like/comment users')
plt.ylabel('cummulative percent')
plt.show()

In [100]:
#count_of_count={unique users number: count of them}
unique, counts = np.unique(count_, return_counts=True)
count_of_count=dict(zip(unique, counts))

In [101]:
count_of_count=sorted(count_of_count.iteritems(), key=lambda d:d[0])


# Import and Transform X

In [4]:
#commall=pd.read_csv("./commall.csv",header=None, names=['time','pageid','uid'],sep=',')
likeall=pd.read_csv("./like-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [6]:
del likeall

In [5]:
commall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
commall['count']=1

In [5]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [16]:
new_metric2=commall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [27]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [13]:
new_metric=new_metric.fillna(value=0)

In [17]:
new_metric2=new_metric2.fillna(value=0)

In [18]:
new_metric_combined=pd.concat([new_metric,new_metric2],axis=1)

In [19]:
new_metric_combined=new_metric_combined.fillna(value=0)

In [24]:
new_metric.shape

(2692, 1303)

In [25]:
new_metric2.shape

(2815, 970)

## Feature Selection (1): Variance

In [21]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.001)
metric_variance=selector.fit_transform(new_metric)

In [30]:
pageid=new_metric_combined.columns[selector.variances_>0.09]

In [22]:
print new_metric.shape
print metric_variance.shape

(2692, 1303)
(2692L, 1145L)


In [31]:
print pageid

Int64Index([      7847926891,       8100113319,       8748127833,
                 10795878252,      12840197743,      13716140500,
                 18809502474,      27743445015,      28211179048,
                 28297822718,
            ...
             522230507809373,  535289793202478,  544241848968882,
             548856635131690,  568718043162016,  569337929781203,
             706305859460576,  761870613900500,  834591649886233,
            1476654739234157],
           dtype='int64', name=u'pageid', length=489)


# Split Test Set

In [27]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_metric, Y, test_size=0.1)
y_train=np.asarray(y_train,dtype="float64")
y_test=np.asarray(y_test,dtype="float64")

In [28]:
print X_test.shape, y_test.shape,X_train.shape,y_train.shape

(270, 1303) (270L, 3L) (2422, 1303) (2422L, 3L)


In [29]:
y_test[:,2].sum()

13.0

In [12]:
X_selected.shape

(1515L, 67L)

## Under-sampling

In [181]:
list=np.where(y_train==0)
len(list[0])

1217

In [182]:
del_index=np.random.choice(list[0],size=len(list[0])/2, replace=False)

In [183]:
y_train=np.delete(y_train,del_index, None)
X_train=np.delete(X_train,del_index, 0)

In [184]:
print y_train.shape
print X_train.shape

(709L,)
(709L, 61L)


## Over-sampling SMOTE

In [338]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind='regular',ratio=0.3)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [284]:
print X_train.shape
print len(y_train[y_train==1])

(3151L, 61L)
727


# PCA

In [101]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
#new_metric=new_metric.fillna(value=0)
metric_pca=pca.fit_transform(X_train)
#metric_pca=pca.fit_transform(pre_metric)

In [146]:
pca.explained_variance_ratio_.sum()

0.58833931652814542

In [862]:
print metric_pca.shape

(1015L, 10L)


# NMF

In [874]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random',alpha=0.01,max_iter=500,tol=1e-4,random_state=0)
model=model.fit(X_train)
metric_nmf=model.transform(X_train)

In [875]:
X_test=model.transform(X_test)

In [35]:
#component-page
model.components_.shape
#user-component
metric_nmf.shape

(1815L, 10L)

###  Assign X_train

In [168]:
X_train=new_metric

In [113]:
X_train=metric_variance

In [97]:
X_train=metric_pca

In [867]:
X_train=metric_nmf

In [868]:
X_train.shape

(1015L, 10L)

## Feature Selection (2) : Coefficient p value

In [37]:
from scipy import stats
from sklearn.linear_model import LogisticRegression
LogReg=LogisticRegression()
LogReg.fit(X_selected, Y_label)
sse = np.sum((LogReg.predict(X_selected) - Y_label) ** 2, axis=0) / float(X_selected.shape[0] - X_selected.shape[1])
se = np.array([
        np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X_selected.T, X_selected))))
    ])
t = LogReg.coef_ / se
p = 2 * (1 - stats.t.cdf(np.abs(t), Y_label.shape[0] - X_selected.shape[1]))
np.set_printoptions(suppress=True)
print 'How many variables are significantly related to Y:',len(p[p<0.05])
print p

How many variables are significantly related to Y: 9
[[ 0.          0.          0.          0.08084395  0.00173334  0.          0.
   0.          0.00021892  0.        ]]


In [38]:
from sklearn.feature_selection import RFE
selector = RFE(LogReg, 9, step=1)
selector = selector.fit(X_selected, Y_label)
selector.support_

array([ True,  True,  True, False,  True,  True,  True,  True,  True,  True], dtype=bool)

In [39]:
X_selected=X_selected[:,selector.support_]
X_selected.shape

(1815L, 9L)

## Feature Selection (3): Chi2

In [55]:
# Chi Square
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [56]:
np.set_printoptions(precision=3,suppress=True)
print 'chi2'
print chi2(metric_variance, Y[:,2])[0]
print 'p value'
print chi2(metric_variance, Y[:,2])[1]

chi2
[ 0.032  0.052  0.857 ...,  0.207  0.786  0.083]
p value
[ 0.857  0.82   0.354 ...,  0.649  0.375  0.774]


In [57]:
#p value
len(chi2(X_train, y_train)[1][chi2(X_train, y_train)[1]<0.05])

782

In [58]:
SKB = SelectKBest(chi2, k=782).fit(X_train, y_train)

In [59]:
X_train=SKB.transform(X_train)
X_test=SKB.transform(X_test)

# Model Comparison

In [14]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_metric, Y, test_size=0.2)
y_train=np.asarray(y_train,dtype="float64")
y_test=np.asarray(y_test,dtype="float64")
X_train=np.asarray(X_train,dtype="float64")
X_test=np.asarray(X_test,dtype="float64")

In [128]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

#cross-validation
from sklearn.cross_validation import KFold
k=10
kf = KFold(len(y_train), n_folds=k,shuffle=True)

classifiers = {"MulBayes": {"f": MultinomialNB()},
               #"RandomForest": {"f": RandomForestClassifier()},
               #"DecisionTree": {"f": DecisionTreeClassifier()},
               #"LogReg": {"f": LogisticRegression()},
               #"LogReg-Reg": {"f": LogisticRegression(C=1.5)},
               "Neural Network": {"f": MLPClassifier(solver='lbfgs', activation="logistic",alpha=1,hidden_layer_sizes=(50,), random_state=1)},
               #"SVC": {"f":SVC(probability=True,kernel='rbf')}
              }

beta=1
for n in range(2,3):
    for model in classifiers.keys():
        recall,rate,precision,f1 = 0.0,0.0,0.0,0.0
        for train_index, test_index in kf:
            # Fit
            classifiers[model]["f"].fit(X_train[train_index], y_train[train_index,n])
            # Predict
            Acc,rate_fold,f1_fold,recall_fold,precision_fold=Score(classifiers[model]["f"].predict(X_train[test_index]),y_train[test_index,n],beta)
            rate+=rate_fold
            f1+=f1_fold
            recall+=recall_fold
            precision+=precision_fold
        rate=float(rate)/k
        recall=float(recall)/k
        precision=float(precision)/k
        f1=float(f1)/k
        classifiers[model]["Scores"]={"accuracy":rate, "f-beta":f1, "recall":recall, "precision":precision}
        classifiers[model]["predict"] = classifiers[model]["f"].predict(X_train)
        classifiers[model]["predict_prob"] = classifiers[model]["f"].predict_proba(X_train)
    
# Evaluate the performance
results = pd.DataFrame(columns=classifiers[classifiers.keys()[0]]["Scores"].keys())

for model in classifiers.keys():
    results.loc[model] = classifiers[model]["Scores"].values()

print results[["accuracy","f-beta","recall","precision"]]

                accuracy    f-beta    recall  precision
Neural Network  0.941012  0.213426  0.215913   0.231429
MulBayes        0.882961  0.304944  0.617619   0.210410


In [119]:
filter=(classifiers["Neural Network"]["f"].coefs_[0]>0.1).sum(axis=1)>0
X_train=X_train[:,filter]

In [120]:
X_train.shape

(2153L, 489L)

In [118]:
sum((classifiers["Neural Network"]["f"].coefs_[0]>0.1).sum(axis=1)>0)

489

# Learning Curve

In [47]:
X=np.asarray(new_metric,dtype="float64")
Y=np.asarray(Y,dtype="float64")

In [125]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
from sklearn.metrics import fbeta_score,make_scorer
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
    title : string. Title for the chart.
    X : array-like, shape (n_samples, n_features).Training vector.
    y : array-like, shape (n_samples) or (n_samples, n_features), optional. None for unsupervised learning.
    ylim : tuple, shape (ymin, ymax), optional
    cv : int, cross-validation generator or an iterable, optional
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.
    n_jobs : integer, optional. Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    fone_scorer = make_scorer(fbeta_score, beta=1)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring=fone_scorer)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


#title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
#cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

#estimator = MultinomialNB()
#plot_learning_curve(estimator, title,X_train, y_train[:,2], ylim=(0, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves (Neural Network using Logistic Neuron)"
cv = ShuffleSplit(n_splits=50, test_size=0.1, random_state=0)
estimator = MLPClassifier(solver='lbfgs', activation="logistic",alpha=0.001,hidden_layer_sizes=(25,), random_state=1)
plot_learning_curve(estimator, title, X_train, y_train[:,2], (0, 1.01), cv=cv, n_jobs=10)

plt.show()

# Test

In [55]:
results = pd.DataFrame(columns=["accuracy","f-beta","recall","precision"])
colors = [ 'yellow', 'blue', 'green']
plt.figure()
lw = 2

for model,color,label in zip(classifiers,colors,labels):
    Acc,rate,f1,recall,precision=Score(classifiers[model]["f"].predict(X_test),y_test,beta)
    results.loc[i]=rate,f1,recall,precision
    fp,tp,threshold=roc_curve(y_test,classifiers[model]["f"].predict_proba(X_test)[:,-1])
    plt.plot(fp, tp, color=color,lw=lw, label=model)
print results
plt.legend(loc="lower right")
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

ValueError: Length mismatch: Expected axis has 5 elements, new values have 3 elements

In [97]:
from sklearn.metrics import roc_curve, auc
results = pd.DataFrame(columns=["accuracy","f-beta","recall","precision"])
colors = [ 'yellow', 'blue', 'green']
labels=[ 'yellow', 'blue', 'local']
plt.figure()
lw = 2
for i,color,label,model in zip(range(3),colors,labels,classifiers):
    Acc,rate,f1,recall,precision=Score(classifiers[model]["f"].predict(X_test),y_test[:,i],beta)
    results.loc[label]=rate,f1,recall,precision
    fp,tp,threshold=roc_curve(y_test[:,i],classifiers[model]["f"].predict_proba(X_test)[:,-1])
    roc_auc = auc(fp, tp)
    plt.plot(fp, tp, color=color,lw=lw, label=label+'(area = %0.2f)' % (roc_auc))

plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',label='Random')
print results
plt.legend(loc="lower right")
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

        accuracy    f-beta    recall  precision
yellow  0.698582  0.628821  0.791209   0.521739
blue    0.808511  0.289474  0.785714   0.177419
local   0.900709  0.333333  0.538462   0.241379
